# How to use Laundromat

#### Import the SpacyModel class. This is the main class that controls Laundromat

In [1]:
from laundromat.spacy.spacy_model import SpacyModel

#### Instantiate your model. The class takes a path to an existing model as an argument. If no argument is given the class defaults to the nb_core_news_lg SpaCy model.

In [2]:
nlp = SpacyModel() 

#### Consider the following text

In [3]:
text = ("Mitt navn er Ola Nordmann. Jeg er 20 år og jobber 50 prosent "
"stilling i Arbeidsgiver AS i Norge. Du kan nå meg på 99999999.")

#### There are a variety of functions that can be applied depending on your needs. Keep in mind that the predict() function returns the start and end position in the form of token index, i.e. which token(s) in the text are part of the entity

In [4]:
#returns [predicted entity, start position, end position, certainty]
entities = nlp.predict(text)
print(entities)

[['Ola Nordmann', 'PER', 3, 5], ['Arbeidsgiver AS', 'ORG', 16, 18], ['Norge', 'LOC', 19, 20]]


#### If you want predictions prettier and no return you can use the display function. It highlights the given text with predicted entities.

In [5]:
nlp.display(text)

In [6]:
nlp.count(text)

{'PER': 1,
 'FNR': 0,
 'DTM': 0,
 'TLF': 0,
 'AMOUNT': 0,
 'LOC': 1,
 'CREDIT_CARD': 0,
 'ORG': 1}

#### If you want to replace the found entities with something, you can use the replace() function. It returns the replaced text. The replacement argument determines which kind of replacement is performed, accepted inputs are entity, character, pad, and shuffle. The default is entity. You can also supply the character to be used for character or pad replacement using the replacement_char argument.

In [7]:
replaced_text_1 = nlp.replace(text, replacement="entity", replacement_char=":^)")
print(replaced_text_1)

Mitt navn er <PER>. Jeg er 20 år og jobber 50 prosent stilling i <ORG> i <LOC>. Du kan nå meg på 99999999.


In [8]:
replaced_text_2 = nlp.replace(text, replacement="pad", replacement_char=":^)")
print(replaced_text_2)

Mitt navn er :^):^):^):^):^):^):^):^):^):^):^):^). Jeg er 20 år og jobber 50 prosent stilling i :^):^):^):^):^):^):^):^):^):^):^):^):^):^):^) i :^):^):^):^):^). Du kan nå meg på 99999999.


In [9]:
replaced_text_3 = nlp.replace(text)
print(replaced_text_3)

Mitt navn er <PER>. Jeg er 20 år og jobber 50 prosent stilling i <ORG> i <LOC>. Du kan nå meg på 99999999.


#### Laundromat also contains the similarity() function which returns the cosine similarity between the unaltered text and the replaced text.

In [10]:
nlp.similarity(replaced_text_1)

0.8880513862428588

In [11]:
nlp.similarity(replaced_text_2)

0.7791217588867314

In [12]:
nlp.similarity(replaced_text_3)

0.8880513862428588

#### The add_patterns() function adds the built-in regex and lookup functionality to the nlp pipeline. Observe how functionality is increased after we call on this function.

In [13]:
nlp.add_patterns()

In [14]:
nlp.predict(text)

[['Ola Nordmann', 'PER', 3, 5],
 ['50 prosent', 'AMOUNT', 12, 14],
 ['Arbeidsgiver AS', 'ORG', 16, 18],
 ['Norge', 'LOC', 19, 20],
 ['99999999.', 'TLF', 26, 27]]

In [15]:
nlp.display(text)

#### You can also disable the NER model if you only wish to use regex and lookup.

In [16]:
nlp.disable_ner()

In [17]:
nlp.display(text)

#### And enable it again

In [18]:
nlp.enable_ner()

In [19]:
nlp.display(text)

#### Now let us explore how to train a model. We shall train a new entity type, but to strengthen performance on already existing entities one just has to provide data with labels corresponding to existing entities. The following "dataset" was copied from SpaCy's website on training new entities. Beware that certain functionality only works for the entity types defined in the code, and so the following example does not inclu

In [20]:
#Note that the index values of the labels are given in string index.
#SpaCy operates with both string index and token index, i.e. which token in the text it is,
# which can be confusing at times.
LABEL = "ANIMAL"

TRAIN_DATA = [
    (
        "Horses are too tall and they pretend to care about your feelings",
        {"entities": [(0, 6, LABEL)]},
    ),
    ("Do they bite?", {"entities": []}),
    (
        "horses are too tall and they pretend to care about your feelings",
        {"entities": [(0, 6, LABEL)]},
    ),
    ("horses pretend to care about your feelings", {"entities": [(0, 6, LABEL)]}),
    (
        "they pretend to care about your feelings, those horses",
        {"entities": [(48, 54, LABEL)]},
    ),
    ("horses?", {"entities": [(0, 6, LABEL)]}),
]

In [21]:
#We see that our model is not capable of recognising horses.
for data in TRAIN_DATA:
    print(nlp.predict(data[0]))

[]
[]
[]
[]
[['those horses', 'PER', 8, 10]]
[]


#### Keep in mind that display() only has functionality for existing entities. It will not display new entities.

#### First let us test our performance on the data. Laundromat contains three scoring functions: SpaCy's built in F1 scorer, a confusion matrix function, and a custom scoring function. The latter two allow for a strict or lax scoring rule, and the custom scoring function prints a variety of metrics.

In [22]:
nlp.f1_scorer(TRAIN_DATA)

{'uas': 0.0,
 'las': 0.0,
 'las_per_type': {'root': {'p': 0.0, 'r': 0.0, 'f': 0.0},
  'flat:foreign': {'p': 0.0, 'r': 0.0, 'f': 0.0},
  'nsubj': {'p': 0.0, 'r': 0.0, 'f': 0.0},
  'flat:name': {'p': 0.0, 'r': 0.0, 'f': 0.0}},
 'ents_p': 0.0,
 'ents_r': 0.0,
 'ents_f': 0.0,
 'ents_per_type': {'ANIMAL': {'p': 0.0, 'r': 0.0, 'f': 0.0},
  'PER': {'p': 0.0, 'r': 0.0, 'f': 0.0}},
 'tags_acc': 0.0,
 'token_acc': 100.0,
 'textcat_score': 0.0,
 'textcats_per_cat': {}}

In [23]:
nlp.confusion_matrix(TRAIN_DATA)

,Is Positive,Is Negative
Predicted Positive,0,1
Predicted Negative,5,41


In [24]:
nlp.print_scores(TRAIN_DATA)

Accuracy is:  0.8723404255319149
Balanced accuracy is:  0.4880952380952381
Precision is:  0.0
Recall is:  0.0
F1 score is:  0.0


#### Now let us train on the available data. The train() function accepts both number of training iterations and output directory as arguments, but these default to 30 and None (which means the model is not saved) respectively.

In [26]:
nlp.train(TRAIN_DATA, labels=[LABEL], n_iter=35)

Losses {'ner': 47.145212272227965}
Losses {'ner': 34.848809380625994}
Losses {'ner': 44.536901545094565}
Losses {'ner': 41.550396441016346}
Losses {'ner': 37.29956804943504}
Losses {'ner': 30.167777620553693}
Losses {'ner': 29.586900326630712}
Losses {'ner': 31.228410091236583}
Losses {'ner': 20.482927678213855}
Losses {'ner': 29.370291504750764}
Losses {'ner': 26.841172312096212}
Losses {'ner': 35.46630643134995}
Losses {'ner': 25.207928742081208}
Losses {'ner': 25.484002753020235}
Losses {'ner': 30.42101531445087}
Losses {'ner': 36.8607010334963}
Losses {'ner': 35.393708864369614}
Losses {'ner': 25.743067639807578}
Losses {'ner': 24.949366394509575}
Losses {'ner': 30.82722828204875}
Losses {'ner': 21.728692426657403}
Losses {'ner': 22.70804280641605}
Losses {'ner': 42.230292881031346}
Losses {'ner': 22.02171700244402}
Losses {'ner': 23.11250706908746}
Losses {'ner': 21.745319498536695}
Losses {'ner': 27.402418407503117}
Losses {'ner': 32.40036462190851}
Losses {'ner': 20.632903023613

In [27]:
nlp.f1_scorer(TRAIN_DATA)

{'uas': 0.0,
 'las': 0.0,
 'las_per_type': {'root': {'p': 0.0, 'r': 0.0, 'f': 0.0},
  'flat:foreign': {'p': 0.0, 'r': 0.0, 'f': 0.0},
  'flat:name': {'p': 0.0, 'r': 0.0, 'f': 0.0},
  'nsubj': {'p': 0.0, 'r': 0.0, 'f': 0.0}},
 'ents_p': 100.0,
 'ents_r': 100.0,
 'ents_f': 100.0,
 'ents_per_type': {'ANIMAL': {'p': 100.0, 'r': 100.0, 'f': 100.0}},
 'tags_acc': 0.0,
 'token_acc': 100.0,
 'textcat_score': 0.0,
 'textcats_per_cat': {}}

In [28]:
nlp.confusion_matrix(TRAIN_DATA)

,Is Positive,Is Negative
Predicted Positive,5,0
Predicted Negative,0,42


In [29]:
nlp.print_scores(TRAIN_DATA)

Accuracy is:  1.0
Balanced accuracy is:  1.0
Precision is:  1.0
Recall is:  1.0
F1 score is:  1.0


In [30]:
for data in TRAIN_DATA:
    print(nlp.predict(data[0]))

[['horses', 'ANIMAL', 0, 1]]
[['Horses', 'ANIMAL', 0, 1]]
[['horses', 'ANIMAL', 9, 10]]
[['horses', 'ANIMAL', 0, 1]]
[['horses', 'ANIMAL', 0, 1]]
[]


#### We observe a strong improvement in  performance. Let us now test it on a new sentence.

In [38]:
horse_text = "What are horses?"

In [39]:
nlp.predict(horse_text)

[['horses', 'ANIMAL', 2, 3]]

In [40]:
nlp.count(text)

{'PER': 1,
 'FNR': 0,
 'DTM': 0,
 'TLF': 1,
 'AMOUNT': 1,
 'LOC': 1,
 'CREDIT_CARD': 0,
 'ORG': 1}

In [42]:
nlp.count("")

{'PER': 0,
 'FNR': 0,
 'DTM': 0,
 'TLF': 0,
 'AMOUNT': 0,
 'LOC': 0,
 'CREDIT_CARD': 0,
 'ORG': 0}